<a href="https://colab.research.google.com/github/angelaoryza/BigData/blob/main/MidTerm_Exam/fp_growth(ver%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRISP-DM Methodology:
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

## 1. Business Understanding

- Find the frequent itemset based on the transaction data.
- Build the recommendation systems.

In [ ]:
# Download the data from the repository

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6' -O market_basket.csv

--2023-04-14 02:15:54--  https://docs.google.com/uc?export=download&id=19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6
Resolving docs.google.com (docs.google.com)... 172.217.219.113, 172.217.219.138, 172.217.219.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.219.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7acqn9v4unui4jqqvp74dujledthm0fn/1681438500000/07765186029080978570/*/19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6?e=download&uuid=f692167e-057c-4822-9db7-3213ce81402f [following]
--2023-04-14 02:15:56--  https://doc-00-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7acqn9v4unui4jqqvp74dujledthm0fn/1681438500000/07765186029080978570/*/19OMIjRgshCsIZvUbpvrdbAKl6O2sRtV6?e=download&uuid=f692167e-057c-4822-9db7-3213ce81402f
Resolving doc-00-88-docs.googleusercontent.com (doc-00-88-docs.googleusercontent.com)... 172.217.219.132, 2607:

# 2. Data Understanding

In [ ]:
# Install Apache Spark
!pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import required libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.fpm import FPGrowth


In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Create Spark Session/Context
spark = SparkSession.builder \
    .master("local") \
    .appName("Frequent Itemset") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","10g") \
    .getOrCreate()

In [ ]:
# Import the data from CSV
df = spark.read.option("header", True).option("delimiter", ";").csv("market_basket.csv")
df.show()

+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|SET 7 BABUSHKA NE...|       2|01.12.2010 08:26| 7,65|     17850|United Kingdom|
|536365|GLASS STAR FROSTE...|       6|01.12.2010 08:26| 4,25|     17850|United Kingdom|
|536366|HAND WARMER UNION...|       6|01.12.2010 08:28| 1,85|     17850|United Kingdom|
|536366|HAND WARMER RED P...|   

## 3. Data Preparation

We need to prepare the data into combination of ["id", "items"] = [1, [a, b, c]]

In [ ]:
# Select only the first two column
df_raw = df.select("BillNo", "Itemname")
df_raw.show()

+------+--------------------+
|BillNo|            Itemname|
+------+--------------------+
|536365|WHITE HANGING HEA...|
|536365| WHITE METAL LANTERN|
|536365|CREAM CUPID HEART...|
|536365|KNITTED UNION FLA...|
|536365|RED WOOLLY HOTTIE...|
|536365|SET 7 BABUSHKA NE...|
|536365|GLASS STAR FROSTE...|
|536366|HAND WARMER UNION...|
|536366|HAND WARMER RED P...|
|536367|ASSORTED COLOUR B...|
|536367|POPPY'S PLAYHOUSE...|
|536367|POPPY'S PLAYHOUSE...|
|536367|FELTCRAFT PRINCES...|
|536367|IVORY KNITTED MUG...|
|536367|BOX OF 6 ASSORTED...|
|536367|BOX OF VINTAGE JI...|
|536367|BOX OF VINTAGE AL...|
|536367|HOME BUILDING BLO...|
|536367|LOVE BUILDING BLO...|
|536367|RECIPE BOX WITH M...|
+------+--------------------+
only showing top 20 rows



In [ ]:
# Before dropping the duplicates
df_raw.count()

522064

In [ ]:
# Remove the duplicates
df_raw = df_raw.dropDuplicates(["BillNo", "Itemname"])
df_raw.count()

511280

In [ ]:
# Convert into this format: ["id", "items"] = [1, [a, b, c]]
# You can use groupBy, agg, and collect_list methods
from pyspark.sql.functions import collect_list
df_input = df_raw.groupBy("BillNo").agg(collect_list('Itemname').alias('items'))
df_input.show()

+------+--------------------+
|BillNo|               items|
+------+--------------------+
|536596|[WAKE UP COCKEREL...|
|538041|                  []|
|538184|[VINTAGE SNAP CAR...|
|538517|[SMALL POPCORN HO...|
|541783|[PLASTERS IN TIN ...|
|544303|[PANTRY ROLLING P...|
|547557|                  []|
|548998|                  []|
|549160|                  []|
|551692|                  []|
|552215|                  []|
|552677|[SET/6 RED SPOTTY...|
|554630|[MONEY BOX POCKET...|
|557271|[ANTIQUE GLASS DR...|
|557757|[SET OF 6 STRAWBE...|
|558700|[FRIDGE MAGNETS L...|
|562463|                  []|
|562890|[SAVOY ART DECO C...|
|562945|                  []|
|566431|[CHRISTMAS TOILET...|
+------+--------------------+
only showing top 20 rows



# 4. Modeling

For the modeling, we are going to use some hyperparameters value and test the RMSES for each model so that we could get the best hyperparameter

## 4.1 Split the data into Training and Testing Set

In [ ]:
training, test = df_input.randomSplit([0.8, 0.2])

## 4.2 Train the training data with each hyperparameter tuning
We are going to use the value `[0.01, 0.05, and 1.00]` for minConfidence and  `[0.01, 0.05, and 1.00]` for minSupport

In [40]:
# Train the data
# 1. Create the instance
#    We can control minSupport and minConfidence variables --> called HYPERPARAMETERS
#    We can use algorithm like Hill Climbing, Grid Search and so on for optimization
#    Or we can also use trial-and-error

#  500.000 data --> minSupport 0.5 --> 250.000 transactions --> IMPOSSIBLE
models = []
minSupport = [0.01, 0.05, 1.00]
minConfidence = [0.01, 0.05, 1.00]

for ms in minSupport:
  for mc in minConfidence:
    fpGrowth = FPGrowth(itemsCol="items", minSupport=ms, minConfidence=mc)
    model_one = fpGrowth.fit(training)
    models.append([model_one, ms, mc])


## 4.3 Predicting the test data and evaluate the SSE

In [41]:
# Display generated association rules.
for model in models:
  model[0].associationRules.show()

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...|0.29516129032258065|14.198077956989248|0.010567650285846279|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...| 0.3387096774193548|  8.60034528426828|0.012126811803430155|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...|0.33548387096774196|11.689284091646655|  0.0120113183576832|
|[LUNCH BAG SUKI D...|[LUNCH BAG RED RE...| 0.6390977443609023| 9.027125317371732|0.014725414332736617|
|[LUNCH BAG SUKI D...|[LUNCH BAG CARS B...|  0.543859649122807|10.605875612454561|0.012531038863544494|
|[LUNCH BAG SUKI D...|[LUNCH BAG SPACEB...| 0.5238095238095238|10.101124191324637|0.012069065080556678|
|[LUNCH BAG SUKI D...|[LUNCH BAG PINK P...|0.48872180451127817| 

🪄 There are only two models that will print results. This is because the next 7 models use too high minimum support score. This means that the rules must be high enough to surpass the threshold to be printed out.

🪄  Since all of the rules have lower minSupport values than the 7 other models' threshold, the result for association rules will be null

🪄  Based on the result, we would choose the model with **minConfidence = 0.05** and **minSupport = 0.01**. The higher the support and confidence level, the more accurate the rules will be

In [47]:
fpGrowth = models[1][0].associationRules

## 5. Evaluation

We take a look into several metrics like min_support, confidence, lift, antencedence and consequent

In [48]:
results = models[1][0].freqItemsets

In [49]:
fpGrowth.show()

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...|0.29516129032258065|14.198077956989248|0.010567650285846279|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...| 0.3387096774193548|  8.60034528426828|0.012126811803430155|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...|0.33548387096774196|11.689284091646655|  0.0120113183576832|
|[LUNCH BAG SUKI D...|[LUNCH BAG RED RE...| 0.6390977443609023| 9.027125317371732|0.014725414332736617|
|[LUNCH BAG SUKI D...|[LUNCH BAG CARS B...|  0.543859649122807|10.605875612454561|0.012531038863544494|
|[LUNCH BAG SUKI D...|[LUNCH BAG SPACEB...| 0.5238095238095238|10.101124191324637|0.012069065080556678|
|[LUNCH BAG SUKI D...|[LUNCH BAG PINK P...|0.48872180451127817| 

In [50]:
results.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[3 STRIPEY MICE F...| 375|
|[T-LIGHT GLASS FL...| 266|
|[DECORATIVE WICKE...| 195|
|[WHITE HANGING HE...|1765|
|[JUMBO BAG RED RE...|1651|
|[JUMBO BAG RED RE...| 346|
|[4 TRADITIONAL SP...| 374|
|[RED RETROSPOT SM...| 266|
|[PAPER BUNTING CO...| 195|
|[REGENCY CAKESTAN...|1523|
|[REGENCY CAKESTAN...| 222|
|[REGENCY CAKESTAN...| 270|
|[NATURAL SLATE CH...| 372|
|[RED RETROSPOT RO...| 265|
|[SKETCHBOOK MAGNE...| 195|
|[DOUGHNUT LIP GLOSS]| 265|
|[RED RETROSPOT CH...| 194|
|     [PARTY BUNTING]|1330|
|[PARTY BUNTING, R...| 297|
|[PARTY BUNTING, J...| 255|
+--------------------+----+
only showing top 20 rows



In [51]:
# Convert the Spark DataFrame back to Pandas DataFrame using Arrow
result_pdf = results.select("*").toPandas()
result_pdf.head()

,items,freq
0,[3 STRIPEY MICE FELTCRAFT],375
1,[T-LIGHT GLASS FLUTED ANTIQUE],266
2,[DECORATIVE WICKER HEART LARGE],195
3,[WHITE HANGING HEART T-LIGHT HOLDER],1765
4,[JUMBO BAG RED RETROSPOT],1651


In [53]:
from google.colab import files
result_pdf.to_excel('result.xlsx')
files.download('result.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
# Display generated association rules.
models[1][0].associationRules.show()

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...|0.29516129032258065|14.198077956989248|0.010567650285846279|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...| 0.3387096774193548|  8.60034528426828|0.012126811803430155|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...|0.33548387096774196|11.689284091646655|  0.0120113183576832|
|[LUNCH BAG SUKI D...|[LUNCH BAG RED RE...| 0.6390977443609023| 9.027125317371732|0.014725414332736617|
|[LUNCH BAG SUKI D...|[LUNCH BAG CARS B...|  0.543859649122807|10.605875612454561|0.012531038863544494|
|[LUNCH BAG SUKI D...|[LUNCH BAG SPACEB...| 0.5238095238095238|10.101124191324637|0.012069065080556678|
|[LUNCH BAG SUKI D...|[LUNCH BAG PINK P...|0.48872180451127817| 

In [56]:
# Display generated association rules.
rules = models[1][0].associationRules

In [57]:
# Convert the Spark DataFrame back to Pandas DataFrame using Arrow
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift,support
0,[HOT WATER BOTTLE KEEP CALM],[LOVE HOT WATER BOTTLE],0.295161,14.198078,0.010568
1,[HOT WATER BOTTLE KEEP CALM],[CHOCOLATE HOT WATER BOTTLE],0.338710,8.600345,0.012127
2,[HOT WATER BOTTLE KEEP CALM],[HOT WATER BOTTLE TEA AND SYMPATHY],0.335484,11.689284,0.012011
3,"[LUNCH BAG SUKI DESIGN, LUNCH BAG BLACK SKULL.]",[LUNCH BAG RED RETROSPOT],0.639098,9.027125,0.014725
4,"[LUNCH BAG SUKI DESIGN, LUNCH BAG BLACK SKULL.]",[LUNCH BAG CARS BLUE],0.543860,10.605876,0.012531


In [58]:
from google.colab import files
result_pdf.to_excel('rules.xlsx')
files.download('rules.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 6. Deployment

In [59]:
# Prepare the input test
test = spark.createDataFrame(
    [
     ("123", ["HAND WARMER RED POLKA DOT"]),
     ("456", ["RED COAT RACK PARIS FASHION"]),
     ("666", ["VINTAGE HEADS AND TAILS CARD GAME", "MINI JIGSAW SPACEBOY"])
    ],
    ["BillNo", "items"]
)

test.show()

+------+--------------------+
|BillNo|               items|
+------+--------------------+
|   123|[HAND WARMER RED ...|
|   456|[RED COAT RACK PA...|
|   666|[VINTAGE HEADS AN...|
+------+--------------------+



In [60]:
# Predict the input based on the model
result = models[1][0].transform(test)
result.show()

+------+--------------------+--------------------+
|BillNo|               items|          prediction|
+------+--------------------+--------------------+
|   123|[HAND WARMER RED ...|                  []|
|   456|[RED COAT RACK PA...|                  []|
|   666|[VINTAGE HEADS AN...|[VINTAGE SNAP CARDS]|
+------+--------------------+--------------------+



In [61]:
# Convert array of values into rows
result = result.select(result.BillNo, result.items, explode(result.prediction).alias("prediction"))
result.show()

+------+--------------------+------------------+
|BillNo|               items|        prediction|
+------+--------------------+------------------+
|   666|[VINTAGE HEADS AN...|VINTAGE SNAP CARDS|
+------+--------------------+------------------+

